In [1]:
from tensorflow import keras
from keras import layers, models, utils
import numpy as np
from utils_fn import generate_dataset, plot_confusion_matrix, plot_history, generate_classification_report, predict_image
from pathlib import Path

utils.set_random_seed(42)

In [2]:
SAVE_NAME = f'Bare model'
DATASET_DIR = Path('Ro_Sign_language_Dataset')
MODEL_SAVE_DIR = Path(f'Saves/{SAVE_NAME}')

BATCH_SIZE = 64
IMAGE_SIZE = (128, 128)
EPOCHS = 50

In [3]:
train_dataset, validation_dataset, test_dataset = generate_dataset(base_dir=DATASET_DIR, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

Found 15600 files belonging to 26 classes.
Found 5200 files belonging to 26 classes.
Found 5200 files belonging to 26 classes.


In [5]:
rescale = layers.Rescaling(1./255)

In [6]:
inputs = keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = rescale(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(26, activation="softmax")(x)
bare_model = keras.Model(inputs=inputs, outputs=outputs)

bare_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                         

In [7]:
bare_model.compile(loss="categorical_crossentropy",
                   optimizer="adam",
                   metrics=["accuracy"])

In [10]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath = str(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.keras'),
                                             save_best_only=True,
                                             monitor="val_loss")]

history = bare_model.fit(train_dataset,
                         epochs=EPOCHS,
                         validation_data=validation_dataset,
                         callbacks=callbacks)

np.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.history.npy', history.history)
bare_model.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.tf', save_format='tf')
bare_model.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.h5', save_format='h5')

Epoch 1/50
244/244 [==============================] - 58s 225ms/step - loss: 1.9450 - accuracy: 0.4095 - val_loss: 1.3530 - val_accuracy: 0.6315
Epoch 2/50
244/244 [==============================] - 11s 44ms/step - loss: 0.2216 - accuracy: 0.9294 - val_loss: 0.7559 - val_accuracy: 0.8146
Epoch 3/50
244/244 [==============================] - 10s 43ms/step - loss: 0.0740 - accuracy: 0.9772 - val_loss: 0.9312 - val_accuracy: 0.7942
Epoch 4/50
244/244 [==============================] - 15s 61ms/step - loss: 0.0343 - accuracy: 0.9899 - val_loss: 0.8517 - val_accuracy: 0.8438
Epoch 5/50
244/244 [==============================] - 14s 57ms/step - loss: 0.0100 - accuracy: 0.9975 - val_loss: 0.8172 - val_accuracy: 0.8508
Epoch 6/50
244/244 [==============================] - 11s 45ms/step - loss: 0.0406 - accuracy: 0.9867 - val_loss: 0.6665 - val_accuracy: 0.8463
Epoch 7/50
244/244 [==============================] - 11s 45ms/step - loss: 0.0288 - accuracy: 0.9924 - val_loss: 0.7517 - val_accuracy

INFO:tensorflow:Assets written to: Saves\Bare model\Bare model.model.tf\assets


INFO:tensorflow:Assets written to: Saves\Bare model\Bare model.model.tf\assets


In [5]:
test_model = models.load_model('Bare model\Bare model.model.keras')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

OSError: Unable to synchronously open file (file signature not found)